<a href="https://colab.research.google.com/github/siddosi/CAIC-Summer-25-IITD/blob/main/AI_DEVCLUB_IITD_TWITTER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df = pd.read_csv("behaviour_simulation_train.csv")  # replace with your actual path

'''print(df.shape)
print(df.tail())
print(df.info())'''
print(df.isnull().sum())



import seaborn as sns
import matplotlib.pyplot as plt

#sns.histplot(df['likes'], kde=True)
#sns.boxplot(x=df['likes'])



df['tweets'] = df['tweets'].str.lower()

print(df['tweets'].head())





print(df['likes'].describe())





word_dict = df['tweets'].str.split().explode().value_counts().to_dict()

from collections import Counter



def calculate_word_probabilities(df, like_thresholds=[10, 20, 100, 1000]):
    results = {}

    for threshold in like_thresholds:
        # Filter tweets with likes > threshold
        filtered_tweets = df[df['likes'] > threshold]['tweets']

        if len(filtered_tweets) == 0:
            results[f'likes_>{threshold}'] = {}
            continue

        # Count words in filtered tweets
        all_words = filtered_tweets.str.split().explode()
        word_counts = Counter(all_words)

        # Calculate total words for probability
        total_words = sum(word_counts.values())

        # Calculate probabilities
        word_probabilities = {word: count/total_words for word, count in word_counts.items()}

        results[f'likes_>{threshold}'] = word_probabilities

    return results

# Use the function
probabilities = calculate_word_probabilities(df)

# Print results
for threshold, probs in probabilities.items():
    print(f"\nWord probabilities for {threshold}:")
    # Show top 10 most probable words
    sorted_probs = dict(sorted(probs.items(), key=lambda x: x[1], reverse=True)[:1000])
    for word, prob in sorted_probs.items():
        if '#' in word:
          print(f"{word}: {prob:.4f}")







id                  0
date                0
likes               0
tweets              0
username            0
media               0
inferred company    0
dtype: int64
0    spend your weekend morning with a ham, egg, an...
1    watch rapper <mention> freestyle for over an h...
2    canadian armenian community demands ban on mil...
3    1st in europe to be devastated by covid-19, it...
4    congratulations to pauletha butts of <mention>...
Name: tweets, dtype: object
count     17331.000000
mean        718.392130
std        3866.475948
min           0.000000
25%           3.000000
50%          73.000000
75%         352.000000
max      254931.000000
Name: likes, dtype: float64

Word probabilities for likes_>10:
#hongkong: 0.0006
#patmcafeeshowlive: 0.0004
#gameon: 0.0003
#f1: 0.0003
#7daysmagicmoment: 0.0003
#antielab: 0.0003
#china: 0.0002
#hongkongprotests: 0.0002
#twd: 0.0002
#cdnpoli: 0.0002
#wearewilliams: 0.0002
#covid19: 0.0002
#ottnews: 0.0002
#microsoftteams: 0.0001
#qatarairways:

In [ ]:
def calculate_word_occurrence_probabilities(df, like_thresholds=[10, 20, 100, 1000]):
    results = {}

    for threshold in like_thresholds:
        # Filter tweets with likes > threshold
        filtered_df = df[df['likes'] > threshold]
        total_tweets = len(filtered_df)

        if total_tweets == 0:
            results[f'likes_>{threshold}'] = {}
            continue

        # Get unique words from all filtered tweets
        all_words = set()
        for tweet in filtered_df['tweets']:
            if pd.notna(tweet):
                all_words.update(tweet.split())

        # Calculate probability of each word appearing in a tweet
        word_probabilities = {}
        for word in all_words:
            # Count tweets containing this word
            tweets_with_word = filtered_df['tweets'].str.contains(rf'\b{word}\b', na=False).sum()
            probability = tweets_with_word / total_tweets
            word_probabilities[word] = probability

        results[f'likes_>{threshold}'] = word_probabilities

    return results

# Use the function
occurrence_probabilities = calculate_word_occurrence_probabilities(df)

error: nothing to repeat at position 2

#TD-IDF

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import math

def calculate_manual_tfidf_with_log(df, like_thresholds=[10, 20, 100, 1000]):
    """
    Calculate TF-IDF manually with logarithmic scaling for different like thresholds
    """
    results = {}

    for threshold in like_thresholds:
        # Filter tweets with likes > threshold
        filtered_df = df[df['likes'] > threshold]

        if len(filtered_df) == 0:
            results[f'likes_>{threshold}'] = {}
            continue

        tweets = filtered_df['tweets'].dropna().tolist()

        if len(tweets) == 0:
            results[f'likes_>{threshold}'] = {}
            continue

        # Get all unique words
        all_words = set()
        for tweet in tweets:
            all_words.update(tweet.split())

        # Calculate TF-IDF for each word
        tfidf_scores = {}

        for word in all_words:
            # Calculate TF (Term Frequency) with log scaling
            tf_scores = []
            for tweet in tweets:
                word_count = tweet.split().count(word)
                total_words = len(tweet.split())

                # TF with log scaling: log(1 + raw_count)
                if word_count > 0:
                    tf = math.log(1 + word_count)
                else:
                    tf = 0
                tf_scores.append(tf)

            # Calculate IDF (Inverse Document Frequency)
            documents_with_word = sum(1 for tweet in tweets if word in tweet.split())
            idf = math.log(len(tweets) / documents_with_word)

            # Calculate TF-IDF for each document and take average
            tfidf_values = [tf * idf for tf in tf_scores]
            avg_tfidf = sum(tfidf_values) / len(tfidf_values) if tfidf_values else 0

            tfidf_scores[word] = {
                'avg_tfidf': avg_tfidf,
                'idf': idf,
                'document_frequency': documents_with_word,
                'total_documents': len(tweets)
            }

        results[f'likes_>{threshold}'] = tfidf_scores

    return results

def calculate_sklearn_tfidf(df, like_thresholds=[10, 20, 100, 1000]):
    """
    Calculate TF-IDF using sklearn's TfidfVectorizer
    """
    results = {}

    for threshold in like_thresholds:
        # Filter tweets with likes > threshold
        filtered_df = df[df['likes'] > threshold]

        if len(filtered_df) == 0:
            results[f'likes_>{threshold}'] = {}
            continue

        tweets = filtered_df['tweets'].dropna().tolist()

        if len(tweets) == 0:
            results[f'likes_>{threshold}'] = {}
            continue

        # Use TfidfVectorizer with log scaling
        vectorizer = TfidfVectorizer(
            lowercase=True,
            stop_words=None,  # Set to 'english' if you want to remove stop words
            use_idf=True,
            smooth_idf=True,
            sublinear_tf=True  # This applies log scaling to TF
        )

        # Fit and transform the tweets
        tfidf_matrix = vectorizer.fit_transform(tweets)
        feature_names = vectorizer.get_feature_names_out()

        # Calculate average TF-IDF scores across all documents
        avg_tfidf_scores = np.mean(tfidf_matrix.toarray(), axis=0)

        # Create dictionary with word: avg_tfidf_score
        word_tfidf = {}
        for i, word in enumerate(feature_names):
            word_tfidf[word] = {
                'avg_tfidf': avg_tfidf_scores[i],
                'idf': vectorizer.idf_[i]
            }

        results[f'likes_>{threshold}'] = word_tfidf

    return results

def display_top_tfidf_words(tfidf_results, top_n=10):
    """
    Display top N words by TF-IDF score for each threshold
    """
    for threshold, words in tfidf_results.items():
        print(f"\n{threshold} - Top {top_n} words by TF-IDF:")
        print("-" * 50)

        if not words:
            print("No data available")
            continue

        # Sort by average TF-IDF score
        sorted_words = sorted(words.items(),
                            key=lambda x: x[1]['avg_tfidf'],
                            reverse=True)[:top_n]

        for word, scores in sorted_words:
          print(f"{word:<15} TF-IDF: {scores['avg_tfidf']:.4f}")

def compare_words_across_thresholds(tfidf_results, words_to_compare):
    """
    Compare specific words across different like thresholds
    """
    print(f"\nComparison of words across thresholds:")
    print("-" * 60)

    # Create header
    thresholds = list(tfidf_results.keys())
    header = f"{'Word':<15}"
    for threshold in thresholds:
        header += f"{threshold:<15}"
    print(header)
    print("-" * 60)

    for word in words_to_compare:
        row = f"{word:<15}"
        for threshold in thresholds:
            if word in tfidf_results[threshold]:
                score = tfidf_results[threshold][word]['avg_tfidf']
                row += f"{score:<15.4f}"
            else:
                row += f"{'N/A':<15}"
        print(row)

# Example usage


sklearn_results = calculate_sklearn_tfidf(df)
display_top_tfidf_words(sklearn_results, top_n=1000)

    # Compare specific words



    # Calculate TF-IDF manually (for understanding)
print("\n" + "="*60)
print("Manual TF-IDF Calculation (for verification)")
print("="*60)








likes_>10 - Top 1000 words by TF-IDF:
--------------------------------------------------
hyperlink       TF-IDF: 0.0530
the             TF-IDF: 0.0433
mention         TF-IDF: 0.0371
to              TF-IDF: 0.0340
in              TF-IDF: 0.0265
and             TF-IDF: 0.0262
of              TF-IDF: 0.0261
for             TF-IDF: 0.0227
on              TF-IDF: 0.0204
you             TF-IDF: 0.0200
is              TF-IDF: 0.0197
with            TF-IDF: 0.0177
this            TF-IDF: 0.0167
it              TF-IDF: 0.0161
we              TF-IDF: 0.0157
at              TF-IDF: 0.0155
your            TF-IDF: 0.0135
from            TF-IDF: 0.0130
are             TF-IDF: 0.0118
our             TF-IDF: 0.0118
that            TF-IDF: 0.0118
new             TF-IDF: 0.0105
all             TF-IDF: 0.0103
be              TF-IDF: 0.0102
more            TF-IDF: 0.0096
now             TF-IDF: 0.0094
by              TF-IDF: 0.0090
as              TF-IDF: 0.0088
here            TF-IDF: 0.0087
what       